In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import re

In [2]:
# Conectar a Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [22]:
# Especifica aquí el nombre de la carpeta en Google Drive
folder_name = "PLANOS"

In [23]:
# Buscar carpeta y obtener ID
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
folder_id = next((f['id'] for f in file_list if f['title'] == folder_name), None)

In [24]:
folder_id

'1A6auWLqyHHUPglxl139hjqNWYprAN9uW'

In [25]:
if folder_id is None:
    raise Exception(f"No se encontró la carpeta '{folder_name}' en Google Drive.")

In [26]:
# Obtener lista de PDFs dentro de la carpeta especificada
query = f"'{folder_id}' in parents and mimeType='application/pdf' and trashed=false"
pdf_files = drive.ListFile({'q': query}).GetList()

In [27]:
pdf_files

[GoogleDriveFile({'kind': 'drive#file', 'userPermission': {'id': 'me', 'type': 'user', 'role': 'owner', 'kind': 'drive#permission', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1zcEJEvdTHW-903CI3fTLpTg7cviNj_TK/permissions/me', 'etag': '"T7IlIsAhbSHCPFye3y0ECi42CNA"', 'pendingOwner': False}, 'fileExtension': 'pdf', 'md5Checksum': '885f33c9db66832fadcd76b0d0adc70e', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1zcEJEvdTHW-903CI3fTLpTg7cviNj_TK', 'ownerNames': ['Bismark sr17'], 'lastModifyingUserName': 'Bismark sr17', 'editable': True, 'writersCanShare': True, 'downloadUrl': 'https://www.googleapis.com/drive/v2/files/1zcEJEvdTHW-903CI3fTLpTg7cviNj_TK?alt=media&source=downloadUrl', 'mimeType': 'application/pdf', 'parents': [{'selfLink': 'https://www.googleapis.com/drive/v2/files/1zcEJEvdTHW-903CI3fTLpTg7cviNj_TK/parents/1A6auWLqyHHUPglxl139hjqNWYprAN9uW', 'id': '1A6auWLqyHHUPglxl139hjqNWYprAN9uW', 'isRoot': False, 'kind': 'drive#parentReference', 'parentLink': 'https:/

In [28]:
# Diccionario para almacenar links
dic_urls = {}

# Extraer links y guardarlos en el diccionario
for file in pdf_files:
    match = re.search(r'_(\d+)\.pdf$', file['title'], re.IGNORECASE)
    if match:
        key = int(match.group(1))
        file.InsertPermission({'type': 'anyone', 'role': 'reader'})  # Asegurar permiso público
        dic_urls[key] = file['alternateLink']
        print(f"Archivo: {file['title']} - Link: {dic_urls[key]}")
    else:
        print(f"Archivo ignorado por formato incorrecto: {file['title']}")

print("\nDiccionario final de URLs generado correctamente.")

Archivo: PLANO_2285.pdf - Link: https://drive.google.com/file/d/1zcEJEvdTHW-903CI3fTLpTg7cviNj_TK/view?usp=drivesdk
Archivo: PLANO_2284.pdf - Link: https://drive.google.com/file/d/1f96_RPq7pXgbO_hfBarFmcB2r2-Kt1ox/view?usp=drivesdk
Archivo: PLANO_2283.pdf - Link: https://drive.google.com/file/d/1Uf3uZ9AFvpONekNqb4JU1FeiemixyVqW/view?usp=drivesdk
Archivo: PLANO_2282.pdf - Link: https://drive.google.com/file/d/1xqHFU8K1YkwNtudhoDk6dqk54EBbqhVj/view?usp=drivesdk
Archivo: PLANO_2280.pdf - Link: https://drive.google.com/file/d/1K1kgkJSnRd0FPZ0xUZr4oC9oDX-h7N_q/view?usp=drivesdk
Archivo: PLANO_2279.pdf - Link: https://drive.google.com/file/d/1KjRNedJlUNtPslkLecy-7iqUZT8sqAQg/view?usp=drivesdk
Archivo: PLANO_2278.pdf - Link: https://drive.google.com/file/d/1zawaAeRsPCps5u_wyC06u3CRCC_kOdGN/view?usp=drivesdk
Archivo: PLANO_2277.pdf - Link: https://drive.google.com/file/d/1dnCbLu2aevWUIGZ905s6FQkiia4tq357/view?usp=drivesdk
Archivo: PLANO_2276.pdf - Link: https://drive.google.com/file/d/1XRHKO6A

In [29]:
len(dic_urls)

1574

In [32]:
import pandas as pd

In [35]:
df_urls = pd.DataFrame(list(dic_urls.items()), columns=['ID', 'URL'])

In [36]:
df_urls.head(3)

,ID,URL
0,2285,https://drive.google.com/file/d/1zcEJEvdTHW-90...
1,2284,https://drive.google.com/file/d/1f96_RPq7pXgbO...
2,2283,https://drive.google.com/file/d/1Uf3uZ9AFvpONe...


In [37]:
df_urls.to_excel('urls_informes.xlsx', index=False)